### This notebook provides a rough idea of how to combine the similar items in the transactions into unified value. This is to be done in order to reduce the item size.

#### Note: So far, i have managed to reduce 5000 different items to approx 3300 items which is considerable difference when it comes to computation costs. This number can be increased / decreased by modifying the hyperparameters like similarity threshold and jaccard token size.



In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#reading the data
df = pd.read_excel('online_merged.xlsx', sheet_name='merged')

In [3]:
# Stripping white spaces from the item description column
df['Description'] = df['Description'].str.strip()

In [4]:
# Defining functions for finding Jaccard similarities

def Intersection(lst1, lst2): 
    return list(set(lst1).intersection(lst2))

def Union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

def tokens(word,token_size = 3):
    return [word[i:i+token_size] for i in range(0,len(word)-token_size + 1)]

def find_jaccard(word1, word2, token_size = 3):
    tokens1 = tokens(word1, token_size)
    tokens2 = tokens(word2, token_size)
    return (len(Intersection(tokens1, tokens2))/len(Union(tokens1, tokens2))) 

In [5]:
# Unique item description list

unq_desc = df.Description.unique()

In [6]:
# Sample list of items to show the working
list1 = ['CHOCOLATE BOX RIBBONS',
'DINOSAUR LUNCH BOX WITH CUTLERY',
'DINOSAUR LUNCHBOX WITH CUTLERY',
'EASTER BUNNY WITH BASKET ON BACK',
'EASTER BUNNY WREATH',
'ELEPHANT TOY WITH BLUE T-SHIRT',
'HEART BUTTONS JEWELLERY BOX',
'LUNCH BOX WITH CUTLERY FAIRY CAKES',
'LUNCH BOX WITH CUTLERY RETROSPOT',
'LUNCHBOX WITH CUTLERY FAIRY CAKES',
'LUNCHBOX WITH CUTLERY RETROSPOT',
'NEW BAROQUE IVORY CUSHION COVER',
'SKULL LUNCH BOX WITH CUTLERY',
'SKULL LUNCHBOX WITH CUTLERY',
'STRAWBERRY LUNCH BOX WITH CUTLERY',
'STRAWBERRY LUNCHBOX WITH CUTLERY',
'UTILTY CABINET WITH HOOKS']
list1

['CHOCOLATE BOX RIBBONS',
 'DINOSAUR LUNCH BOX WITH CUTLERY',
 'DINOSAUR LUNCHBOX WITH CUTLERY',
 'EASTER BUNNY WITH BASKET ON BACK',
 'EASTER BUNNY WREATH',
 'ELEPHANT TOY WITH BLUE T-SHIRT',
 'HEART BUTTONS JEWELLERY BOX',
 'LUNCH BOX WITH CUTLERY FAIRY CAKES',
 'LUNCH BOX WITH CUTLERY RETROSPOT',
 'LUNCHBOX WITH CUTLERY FAIRY CAKES',
 'LUNCHBOX WITH CUTLERY RETROSPOT',
 'NEW BAROQUE IVORY CUSHION COVER',
 'SKULL LUNCH BOX WITH CUTLERY',
 'SKULL LUNCHBOX WITH CUTLERY',
 'STRAWBERRY LUNCH BOX WITH CUTLERY',
 'STRAWBERRY LUNCHBOX WITH CUTLERY',
 'UTILTY CABINET WITH HOOKS']

In [7]:
len(list1)

17

In [8]:
# Grouping the similar items based on unique item description list to create a lookup result

from difflib import SequenceMatcher as sm

combined_list = list1 # Use unq_desc to get complete list of combinations from txns.
result = list()
result_group = list()
usedElements = list()
skip = False

for firstName in combined_list:
    skip = False

    for x in usedElements:
        if x == firstName:
            skip = True
    if skip == True:
        continue

    for secondName in combined_list:

        #if(sm(None, firstName, secondName).ratio() >= 0.70):            
        if(find_jaccard(firstName, secondName) >= 0.50):            

            result_group.append(secondName)
            usedElements.append(secondName)
        else:
            pass

    result.append(result_group[:])
    del result_group[:]

for value in result:
    print(value)

['CHOCOLATE BOX RIBBONS']
['DINOSAUR LUNCH BOX WITH CUTLERY', 'DINOSAUR LUNCHBOX WITH CUTLERY', 'LUNCH BOX WITH CUTLERY FAIRY CAKES', 'LUNCH BOX WITH CUTLERY RETROSPOT', 'SKULL LUNCH BOX WITH CUTLERY', 'SKULL LUNCHBOX WITH CUTLERY', 'STRAWBERRY LUNCH BOX WITH CUTLERY']
['EASTER BUNNY WITH BASKET ON BACK']
['EASTER BUNNY WREATH']
['ELEPHANT TOY WITH BLUE T-SHIRT']
['HEART BUTTONS JEWELLERY BOX']
['LUNCH BOX WITH CUTLERY FAIRY CAKES', 'LUNCHBOX WITH CUTLERY FAIRY CAKES', 'LUNCHBOX WITH CUTLERY RETROSPOT', 'SKULL LUNCHBOX WITH CUTLERY', 'STRAWBERRY LUNCHBOX WITH CUTLERY']
['NEW BAROQUE IVORY CUSHION COVER']
['UTILTY CABINET WITH HOOKS']


In [9]:
df1 = df
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,493414,21844,RETRO SPOT MUG,36,2010-01-04 10:28:00,2.55,14590,United Kingdom
1,493414,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-04 10:28:00,4.25,14590,United Kingdom
2,493414,37508,NEW ENGLAND CERAMIC CAKE SERVER,2,2010-01-04 10:28:00,2.55,14590,United Kingdom
3,493414,35001G,HAND OPEN SHAPE GOLD,2,2010-01-04 10:28:00,4.25,14590,United Kingdom
4,493414,21527,RETRO SPOT TRADITIONAL TEAPOT,12,2010-01-04 10:28:00,6.95,14590,United Kingdom


In [10]:
df1_price = df1[(df1[['Price','Quantity']] > 0).all(axis=1)]
df1.describe()

,Invoice,Quantity,Price,Customer ID
count,770692.000000,770692.000000,770692.000000,770692.000000
mean,539271.351953,13.286099,2.993243,15327.973197
std,25509.968171,146.100973,11.160934,1696.651545
min,493414.000000,1.000000,0.001000,12346.000000
25%,517964.000000,2.000000,1.250000,13975.000000
50%,537868.000000,5.000000,1.950000,15260.000000
75%,562460.000000,12.000000,3.750000,16800.000000
max,581587.000000,80995.000000,8142.750000,18287.000000


In [11]:
# Creating a sample transaction dataframe from origional DF for testing purposes.

#np.random.seed(123)
#sample_desc = np.random.choice(unq_desc, size = 7)

sample_txn = df1[df1['Description'].isin(list1)]

sample_txn = sample_txn.sample(n = 100)

len(sample_txn.Invoice.unique())

sample_txn.groupby(['StockCode']).count()

sample_txn['TotalPrice'] = sample_txn['Price']*sample_txn['Quantity']

sample_txn

#basket = (sample_txn.groupby(['Invoice', 'Description'])['TotalPrice'] 
#          .sum().unstack().reset_index().fillna(0) 
#          .set_index('Invoice')) 
#basket

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
11081,494966,22352,LUNCHBOX WITH CUTLERY RETROSPOT,5,2010-01-20 11:01:00,2.55,14680,United Kingdom,12.75
220360,521471,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,5,2010-09-06 12:09:00,2.55,14680,United Kingdom,12.75
304561,530567,21558,SKULL LUNCH BOX WITH CUTLERY,6,2010-11-03 14:00:00,2.55,17254,United Kingdom,15.30
768064,581310,22352,LUNCH BOX WITH CUTLERY RETROSPOT,1,2011-12-08 11:43:00,2.55,16442,United Kingdom,2.55
206146,519694,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,10,2010-08-19 14:11:00,2.55,13089,United Kingdom,25.50
...,...,...,...,...,...,...,...,...,...
39223,498721,21561,DINOSAUR LUNCHBOX WITH CUTLERY,6,2010-02-22 14:34:00,2.55,13178,United Kingdom,15.30
496336,552261,22352,LUNCH BOX WITH CUTLERY RETROSPOT,12,2011-05-08 11:34:00,2.55,16807,United Kingdom,30.60
619706,567179,85049G,CHOCOLATE BOX RIBBONS,1,2011-09-18 14:27:00,1.25,15547,United Kingdom,1.25
217227,521038,21558,SKULL LUNCH BOX WITH CUTLERY,6,2010-09-02 09:14:00,2.55,12589,France,15.30


In [15]:
before_len = len(sample_txn['Description'].unique())
before_len

12

In [16]:
# Checking the groupped values before unifying the descriptions

before_sums = sample_txn.groupby(['Description']).sum()
before_sums

,Invoice,Quantity,Price,Customer ID,TotalPrice
Description,,,,,
CHOCOLATE BOX RIBBONS,5408447,33,12.50,153779,41.25
DINOSAUR LUNCH BOX WITH CUTLERY,4725893,87,22.50,138861,194.85
DINOSAUR LUNCHBOX WITH CUTLERY,1997625,14,10.20,60880,35.70
EASTER BUNNY WITH BASKET ON BACK,994369,16,1.30,31492,10.40
ELEPHANT TOY WITH BLUE T-SHIRT,496927,1,3.75,17293,3.75
LUNCH BOX WITH CUTLERY FAIRY CAKES,3110669,94,14.85,85582,212.70
LUNCH BOX WITH CUTLERY RETROSPOT,11824864,352,54.75,329724,789.60
LUNCHBOX WITH CUTLERY FAIRY CAKES,2501632,44,12.75,73780,112.20
LUNCHBOX WITH CUTLERY RETROSPOT,2980836,39,15.30,89418,99.45


In [17]:
#unifying the item description using lookup from results

for index, row in sample_txn.iterrows():
    
    for desc_list in result:
            #if col in 
        if(row['Description'] in desc_list):
            print('before conversion:')
            print(row['Description'], ',', desc_list[0])
            sample_txn.at[index,'Description'] =  desc_list[0]
            print('After conversion:')
            print(row['Description'], ',', desc_list[0])
            print('-------------------------------------------------------------------------')
            
            break
            #print('-------------------------------------------------------------------------')
            #print(round(row.sum(),2))
#col_list_full    

before conversion:
LUNCHBOX WITH CUTLERY RETROSPOT , LUNCH BOX WITH CUTLERY FAIRY CAKES
After conversion:
LUNCHBOX WITH CUTLERY RETROSPOT , LUNCH BOX WITH CUTLERY FAIRY CAKES
-------------------------------------------------------------------------
before conversion:
STRAWBERRY LUNCH BOX WITH CUTLERY , DINOSAUR LUNCH BOX WITH CUTLERY
After conversion:
STRAWBERRY LUNCH BOX WITH CUTLERY , DINOSAUR LUNCH BOX WITH CUTLERY
-------------------------------------------------------------------------
before conversion:
SKULL LUNCH BOX WITH CUTLERY , DINOSAUR LUNCH BOX WITH CUTLERY
After conversion:
SKULL LUNCH BOX WITH CUTLERY , DINOSAUR LUNCH BOX WITH CUTLERY
-------------------------------------------------------------------------
before conversion:
LUNCH BOX WITH CUTLERY RETROSPOT , DINOSAUR LUNCH BOX WITH CUTLERY
After conversion:
LUNCH BOX WITH CUTLERY RETROSPOT , DINOSAUR LUNCH BOX WITH CUTLERY
-------------------------------------------------------------------------
before conversion:
STR

In [18]:
# Checking the groupped values after unifying the descriptions

after_sums = sample_txn.groupby(['Description']).sum()
after_sums

,Invoice,Quantity,Price,Customer ID,TotalPrice
Description,,,,,
CHOCOLATE BOX RIBBONS,5408447,33,12.50,153779,41.25
DINOSAUR LUNCH BOX WITH CUTLERY,38937655,759,183.45,1088060,1746.45
EASTER BUNNY WITH BASKET ON BACK,994369,16,1.30,31492,10.40
ELEPHANT TOY WITH BLUE T-SHIRT,496927,1,3.75,17293,3.75
LUNCH BOX WITH CUTLERY FAIRY CAKES,6979849,96,35.70,205957,244.80


In [19]:
after_len = len(sample_txn['Description'].unique())
after_len

5

In [20]:
basket = (sample_txn.groupby(['Invoice', 'Description'])['TotalPrice'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Invoice')) 
basket

Description,CHOCOLATE BOX RIBBONS,DINOSAUR LUNCH BOX WITH CUTLERY,EASTER BUNNY WITH BASKET ON BACK,ELEPHANT TOY WITH BLUE T-SHIRT,LUNCH BOX WITH CUTLERY FAIRY CAKES
Invoice,,,,,
494803,0.0,0.00,0.0,0.0,2.55
494897,0.0,0.00,0.0,0.0,30.60
494966,0.0,0.00,0.0,0.0,12.75
495682,0.0,0.00,0.0,0.0,15.30
495706,0.0,2.55,0.0,0.0,0.00
...,...,...,...,...,...
571243,0.0,15.30,0.0,0.0,0.00
572294,0.0,5.10,0.0,0.0,0.00
573002,0.0,126.00,0.0,0.0,0.00


In [21]:
list2 = list(basket.columns)

In [22]:
import itertools
subsets = []
stuff = list2
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        subsets.append(list(subset))

subsets
    

[[],
 ['CHOCOLATE BOX RIBBONS'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['EASTER BUNNY WITH BASKET ON BACK'],
 ['ELEPHANT TOY WITH BLUE T-SHIRT'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['CHOCOLATE BOX RIBBONS', 'DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['CHOCOLATE BOX RIBBONS', 'EASTER BUNNY WITH BASKET ON BACK'],
 ['CHOCOLATE BOX RIBBONS', 'ELEPHANT TOY WITH BLUE T-SHIRT'],
 ['CHOCOLATE BOX RIBBONS', 'LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY', 'EASTER BUNNY WITH BASKET ON BACK'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY', 'ELEPHANT TOY WITH BLUE T-SHIRT'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY', 'LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['EASTER BUNNY WITH BASKET ON BACK', 'ELEPHANT TOY WITH BLUE T-SHIRT'],
 ['EASTER BUNNY WITH BASKET ON BACK', 'LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['ELEPHANT TOY WITH BLUE T-SHIRT', 'LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['CHOCOLATE BOX RIBBONS',
  'DINOSAUR LUNCH BOX WITH CUTLERY',
  'EASTER BUNNY WITH BASKET ON BACK'],
 ['CHOC

In [23]:

col_list_full = []
for index, row in basket.iterrows():
    col_list = []
    for col in basket:
            #if col in 
        if(basket.at[index,col]!=0.0):
            col_list.append(col)
    col_list_full.append(col_list)
    #print('-------------------------------------------------------------------------')
            #print(round(row.sum(),2))
col_list_full  

[['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['EASTER BUNNY WITH BASKET ON BACK'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['ELEPHANT TOY WITH BLUE T-SHIRT'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['EASTER BUNNY WITH BASKET ON BACK'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['LUNCH BOX WITH CUTLERY FAIRY CAKES'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['CHOCOLATE BOX RIBBONS'],
 ['DINOSAUR LUNCH BOX WITH CUTLERY'],
 ['DINOSAUR LUNCH

In [25]:
# grouping the invoice and columns based on the subsets

#Finding invoice wise sum

invoice_sum = list(basket.sum(axis = 1))
invoice_sum

before_grp = []
for i in range (len(basket)):
    temp = [str(col_list_full[i]),invoice_sum[i]]
    #print(col_list_full[i], ' ',invoice_sum[i])
    before_grp.append(temp)

before_grp

e_dataframe = pd.DataFrame(before_grp)
e_dataframe.columns = ['a', 'b']

e_dataframe

groupped_sum = pd.DataFrame(e_dataframe.groupby(['a']).sum())
groupped_sum.reset_index(level=0, inplace=True)
groupped_sum.columns = ['a', 'b']

In [26]:
groupped_sum

,a,b
0,['CHOCOLATE BOX RIBBONS'],41.25
1,['DINOSAUR LUNCH BOX WITH CUTLERY'],1746.45
2,['EASTER BUNNY WITH BASKET ON BACK'],10.40
3,['ELEPHANT TOY WITH BLUE T-SHIRT'],3.75
4,['LUNCH BOX WITH CUTLERY FAIRY CAKES'],244.80


In [27]:
# finding characteristic function value for each group and assigning them to subset list
all_sums = []
for sub in subsets:
    sum1 = 0
    print('subset id:', subsets.index(sub))
    print("step1")
    #print(sub)
    print('------------')
    stuff = sub
    new_sub = []
    for L in range(1, len(stuff)+1):
        for subset in itertools.combinations(stuff, L):
            new_sub.append(list(subset))
    print(new_sub)
    for item in new_sub:
        index = groupped_sum[groupped_sum['a'] == str(item)].index
        if index.empty:
            print('None')
        else:
            print(item, index)
            sum1+=groupped_sum['b'][index].values[0]
    all_sums.append(sum1)
    print('------------------------------------------------------------')

subset id: 0
step1
------------
[]
------------------------------------------------------------
subset id: 1
step1
------------
[['CHOCOLATE BOX RIBBONS']]
['CHOCOLATE BOX RIBBONS'] Int64Index([0], dtype='int64')
------------------------------------------------------------
subset id: 2
step1
------------
[['DINOSAUR LUNCH BOX WITH CUTLERY']]
['DINOSAUR LUNCH BOX WITH CUTLERY'] Int64Index([1], dtype='int64')
------------------------------------------------------------
subset id: 3
step1
------------
[['EASTER BUNNY WITH BASKET ON BACK']]
['EASTER BUNNY WITH BASKET ON BACK'] Int64Index([2], dtype='int64')
------------------------------------------------------------
subset id: 4
step1
------------
[['ELEPHANT TOY WITH BLUE T-SHIRT']]
['ELEPHANT TOY WITH BLUE T-SHIRT'] Int64Index([3], dtype='int64')
------------------------------------------------------------
subset id: 5
step1
------------
[['LUNCH BOX WITH CUTLERY FAIRY CAKES']]
['LUNCH BOX WITH CUTLERY FAIRY CAKES'] Int64Index([4], dtyp

None
------------------------------------------------------------
subset id: 30
step1
------------
[['DINOSAUR LUNCH BOX WITH CUTLERY'], ['EASTER BUNNY WITH BASKET ON BACK'], ['ELEPHANT TOY WITH BLUE T-SHIRT'], ['LUNCH BOX WITH CUTLERY FAIRY CAKES'], ['DINOSAUR LUNCH BOX WITH CUTLERY', 'EASTER BUNNY WITH BASKET ON BACK'], ['DINOSAUR LUNCH BOX WITH CUTLERY', 'ELEPHANT TOY WITH BLUE T-SHIRT'], ['DINOSAUR LUNCH BOX WITH CUTLERY', 'LUNCH BOX WITH CUTLERY FAIRY CAKES'], ['EASTER BUNNY WITH BASKET ON BACK', 'ELEPHANT TOY WITH BLUE T-SHIRT'], ['EASTER BUNNY WITH BASKET ON BACK', 'LUNCH BOX WITH CUTLERY FAIRY CAKES'], ['ELEPHANT TOY WITH BLUE T-SHIRT', 'LUNCH BOX WITH CUTLERY FAIRY CAKES'], ['DINOSAUR LUNCH BOX WITH CUTLERY', 'EASTER BUNNY WITH BASKET ON BACK', 'ELEPHANT TOY WITH BLUE T-SHIRT'], ['DINOSAUR LUNCH BOX WITH CUTLERY', 'EASTER BUNNY WITH BASKET ON BACK', 'LUNCH BOX WITH CUTLERY FAIRY CAKES'], ['DINOSAUR LUNCH BOX WITH CUTLERY', 'ELEPHANT TOY WITH BLUE T-SHIRT', 'LUNCH BOX WITH CUTL

In [28]:
all_sums

[0,
 41.25,
 1746.449999999999,
 10.4,
 3.75,
 244.80000000000004,
 1787.699999999999,
 51.65,
 45.0,
 286.05000000000007,
 1756.849999999999,
 1750.199999999999,
 1991.2499999999989,
 14.15,
 255.20000000000005,
 248.55000000000004,
 1798.099999999999,
 1791.449999999999,
 2032.4999999999989,
 55.4,
 296.45000000000005,
 289.80000000000007,
 1760.599999999999,
 2001.649999999999,
 1994.9999999999989,
 258.95000000000005,
 1801.849999999999,
 2042.899999999999,
 2036.2499999999989,
 300.20000000000005,
 2005.399999999999,
 2046.649999999999]

In [29]:
for i in range(len(all_sums)):
    print(subsets[i], " ", all_sums[i])

[]   0
['CHOCOLATE BOX RIBBONS']   41.25
['DINOSAUR LUNCH BOX WITH CUTLERY']   1746.449999999999
['EASTER BUNNY WITH BASKET ON BACK']   10.4
['ELEPHANT TOY WITH BLUE T-SHIRT']   3.75
['LUNCH BOX WITH CUTLERY FAIRY CAKES']   244.80000000000004
['CHOCOLATE BOX RIBBONS', 'DINOSAUR LUNCH BOX WITH CUTLERY']   1787.699999999999
['CHOCOLATE BOX RIBBONS', 'EASTER BUNNY WITH BASKET ON BACK']   51.65
['CHOCOLATE BOX RIBBONS', 'ELEPHANT TOY WITH BLUE T-SHIRT']   45.0
['CHOCOLATE BOX RIBBONS', 'LUNCH BOX WITH CUTLERY FAIRY CAKES']   286.05000000000007
['DINOSAUR LUNCH BOX WITH CUTLERY', 'EASTER BUNNY WITH BASKET ON BACK']   1756.849999999999
['DINOSAUR LUNCH BOX WITH CUTLERY', 'ELEPHANT TOY WITH BLUE T-SHIRT']   1750.199999999999
['DINOSAUR LUNCH BOX WITH CUTLERY', 'LUNCH BOX WITH CUTLERY FAIRY CAKES']   1991.2499999999989
['EASTER BUNNY WITH BASKET ON BACK', 'ELEPHANT TOY WITH BLUE T-SHIRT']   14.15
['EASTER BUNNY WITH BASKET ON BACK', 'LUNCH BOX WITH CUTLERY FAIRY CAKES']   255.20000000000005
['